# Dataloader de BERTMHC

En este script, se detalla el dataloader para preparar el training set. EL training set de ejemplo esta en el archivo train_old.csv
De dicho archivo, se utiliza la columna 'mhc' y 'peptide' concatenadas como input y el target esta compuesto por las columnas 'label' y 'masslabel'. Tambien se utiliza un la función collate_fn de pytorch para asegurar el mismo tamaño de los inputs.



In [3]:

from typing import Union, List, Tuple, Sequence, Dict, Any, Optional, Collection, Mapping
from pathlib import Path
from tape.tokenizers import TAPETokenizer
from tape.datasets import pad_sequences as tape_pad
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import torch

from torch.utils.data import DataLoader
import logging

Trainset

In [4]:
class CSVDataset(Dataset):
    def __init__(self,
                 data_file: Union[str, Path, pd.DataFrame],
                 max_pep_len=30,
                 train: bool = True):
        if isinstance(data_file, pd.DataFrame):
            data = data_file
        else:
            data = pd.read_csv(data_file)
        mhc = data['mhc']
        self.mhc = mhc.values
        peptide = data['peptide']
        peptide = peptide.apply(lambda x: x[:max_pep_len])
        self.peptide = peptide.values
        if not train:
            data['label'] = np.nan
            data['masslabel'] = np.nan
        if 'masslabel' not in data and 'label' not in data:
            raise ValueError("missing label.")
        if 'masslabel' not in data:
            data['masslabel'] = np.nan
        if 'label' not in data:
            data['label'] = np.nan

        ###########################################################################################################
        ##### el target esta compuesto por el label(float) y masslabel(int) #######################################
        self.targets = np.stack([data['label'], data['masslabel']], axis=1)
        self.data = data        

    def __len__(self) -> int:
        return len(self.mhc)

    def __getitem__(self, index: int):
        ###########################################################################################################
        ##### aqui concatena el MHC con el peptido para que todo eso sea el input #################################
        seq = self.mhc[index] + self.peptide[index]
        return {
            "id": str(index),
            "primary": seq,
            "protein_length": len(seq),
            "targets": self.targets[index]}

In [5]:
class BertDataset(Dataset):
    ''' Load data for pretrained Bert model, implemented in TAPE
    '''

    def __init__(self,
                 input_file,
                 tokenizer: Union[str, TAPETokenizer] = 'iupac',
                 max_pep_len=30,
                 train: bool = True):
        if isinstance(tokenizer, str):
            tokenizer = TAPETokenizer(vocab=tokenizer)
        self.tokenizer = tokenizer
        self.data = CSVDataset(input_file,
                               max_pep_len=max_pep_len,
                               train=train)        

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int):
        item = self.data[index]
        
        token_ids = self.tokenizer.encode(item['primary'])
        input_mask = np.ones_like(token_ids)
        ret = {'input_ids': token_ids,
               'input_mask': input_mask,
               'targets': item['targets']}
        
        return ret

    def collate_fn(self, batch) -> Dict[str, torch.Tensor]:
        elem = batch[0]
        batch = {key: [d[key] for d in batch] for key in elem}
        input_ids = torch.from_numpy(tape_pad(batch['input_ids'], 0))
        input_mask = torch.from_numpy(tape_pad(batch['input_mask'], 0))
        tmp = np.array(batch['targets'])
        #targets = torch.tensor(batch['targets'], dtype=torch.float32)
        targets = torch.tensor(tmp, dtype=torch.float32)
        ret = {'input_ids': input_ids,
               'input_mask': input_mask,
               'targets': targets}
        
        return ret


In [6]:
trainset = BertDataset('../../dataset/netMHCIIpan3.2/train_mini.csv', max_pep_len=24, instance_weight=False)
valset = BertDataset('../../dataset/netMHCIIpan3.2/eval_mini.csv', max_pep_len=24, instance_weight=False)
first_sample = trainset[0] 
print(first_sample['input_ids']) # indices del one-hot encoding
print(first_sample['input_mask'])
print(first_sample['targets']) 

[ 2 20  9 10 10 13  5 22 11  5  5 25  8  5 13 16  9 22 22 10  8 28 10  8
 13  8  9  5 23 28 12 25 25 10 23 23 13 19 15 25  5 15 23 15 23 22 28 15
 11 15 14  3]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0.698876 1.      ]


Train data

In [7]:
logging.basicConfig(format='%(name)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

train_data = DataLoader(        trainset,
                                batch_size=32,
                                shuffle=True,
                                num_workers=16,
                                pin_memory=True,
                                collate_fn=trainset.collate_fn)

val_data = DataLoader(        valset,
                              batch_size=64,
                              num_workers=16,
                              pin_memory=True,
                              collate_fn=valset.collate_fn)

logger.info("Training on {0} samples, eval on {1}".format(len(trainset), len(valset)))

__main__ - Training on 107424 samples, eval on 13428


# Train BERTMHC

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from bertmhc import BERTMHC
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utils_model import EarlyStopping
from utils_model import train, evaluate

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device", device)

print("\nCargamos los pesos de TAPE\n\n")
model = BERTMHC.from_pretrained('bert-base')

for p in model.bert.parameters():
    p.requires_grad = True

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model = model.to(device)

Device cuda

Cargamos los pesos de TAPE




tape.models.modeling_utils - loading configuration file https://s3.amazonaws.com/songlabdata/proteindata/pytorch-models/bert-base-config.json from cache at /home/vicente/.cache/torch/protein_models/fbb05edff0ffa844a729a04850272a1f8973bc002526f6615ad113a5f5aacd36.05edb4ed225e1907a3878f9d68b275d79e025b667555aa94a086e27cb5c591e0
tape.models.modeling_utils - Model config {
  "attention_probs_dropout_prob": 0.1,
  "base_model": "transformer",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "input_size": 768,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 8192,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_size": 768,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 30
}

tape.models.modeling_utils - loading weights file https://s3.a

Parametros de BERTMHC

TAPE tiene 92356612 parametros <br>
ProtTrans (prot_bert_bfd) tiene 419933186 (4x larger)

In [7]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

92356612


Training

In [8]:
epochs = 10
lr = 0.15
w_pos = 1.0 # mass positive weight
save = "bertmhc_model.pt"
alpha = 0.0 # alpha weight on mass loss, affinity loss weight with 1-alpha
patience = 5 # Earlystopping patience
metric = 'val_auc' # validation metric, default auc

aff_criterion = nn.BCEWithLogitsLoss()
w_pos = torch.tensor([w_pos]).to(device)
mass_criterion = nn.BCEWithLogitsLoss(pos_weight=w_pos, reduction='none')

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=2, min_lr=1e-4, factor=0.1)

early_stopping = EarlyStopping(patience=patience, verbose=True, saveto=save)

for epoch in range(epochs):
    print("Training epoch {}".format(epoch))
    train_metrics = train(model, optimizer, train_data, device, aff_criterion, mass_criterion, alpha, scheduler)
    eval_metrics = evaluate(model, val_data, device, aff_criterion, mass_criterion, alpha)
    eval_metrics['train_loss'] = train_metrics
    logs = eval_metrics

    scheduler.step(logs.get(metric))
    logging.info('Sample dict log: %s' % logs)

    # callbacks
    early_stopping(-logs.get(metric), model, optimizer)
    if early_stopping.early_stop or logs.get(metric) <= 0:
        print("Early stopping")
        break

Training epoch 0


100%|██████████| 3357/3357 [08:12<00:00,  6.81it/s]
root - Sample dict log: {'val_cor': 0.5033360818216253, 'val_auc': 0.8437104698380997, 'val_ap': 0.28102128923439507, 'val_mass_auc': 0.2372045758505762, 'val_loss': 0.009377251796719572, 'train_loss': 0.018801003750574564}


Validation loss (inf --> -0.843710).  Saving model ...
Training epoch 1


  3%|▎         | 91/3357 [00:14<08:37,  6.31it/s]


KeyboardInterrupt: 

: 

# Train ImmunoBERT

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from bertmhc import TAPEBackbone
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utils_model import EarlyStopping
from utils_model import train, evaluate

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device", device)

print("\nCargamos los pesos de TAPE\n\n")
model = TAPEBackbone.from_pretrained('bert-base')
model = model.to(device)


Device cuda

Cargamos los pesos de TAPE




tape.models.modeling_utils - loading configuration file https://s3.amazonaws.com/songlabdata/proteindata/pytorch-models/bert-base-config.json from cache at /home/vicente/.cache/torch/protein_models/fbb05edff0ffa844a729a04850272a1f8973bc002526f6615ad113a5f5aacd36.05edb4ed225e1907a3878f9d68b275d79e025b667555aa94a086e27cb5c591e0
tape.models.modeling_utils - Model config {
  "attention_probs_dropout_prob": 0.1,
  "base_model": "transformer",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "input_size": 768,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 8192,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_size": 768,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "vocab_size": 30
}

tape.models.modeling_utils - loading weights file https://s3.a

In [9]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

91961856


In [10]:
epochs = 10
lr = 0.15
w_pos = 1.0 # mass positive weight
save = "bertmhc_model.pt"
alpha = 0.0 # alpha weight on mass loss, affinity loss weight with 1-alpha
patience = 5 # Earlystopping patience
metric = 'val_auc' # validation metric, default auc

aff_criterion = nn.BCEWithLogitsLoss()
w_pos = torch.tensor([w_pos]).to(device)
mass_criterion = nn.BCEWithLogitsLoss(pos_weight=w_pos, reduction='none')

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=2, min_lr=1e-4, factor=0.1)

early_stopping = EarlyStopping(patience=patience, verbose=True, saveto=save)

for epoch in range(epochs):
    print("Training epoch {}".format(epoch))
    train_metrics = train(model, optimizer, train_data, device, aff_criterion, mass_criterion, alpha, scheduler)
    eval_metrics = evaluate(model, val_data, device, aff_criterion, mass_criterion, alpha)
    eval_metrics['train_loss'] = train_metrics
    logs = eval_metrics

    scheduler.step(logs.get(metric))
    logging.info('Sample dict log: %s' % logs)

    # callbacks
    early_stopping(-logs.get(metric), model, optimizer)
    if early_stopping.early_stop or logs.get(metric) <= 0:
        print("Early stopping")
        break

Training epoch 0


  0%|          | 0/3357 [00:00<?, ?it/s]


TypeError: forward() missing 2 required positional arguments: 'token_type_ids' and 'position_ids'